<a href="https://colab.research.google.com/github/youiba/Tensorflow_Optimisation_Techniques/blob/main/Optimization_Techniques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install -q tensorflow
!pip3 install -q tensorflow-model-optimization

     |████████████████████████████████| 238 kB 33.0 MB/s 


In [ ]:
import tempfile
import os
import zipfile
import tensorflow as tf
from tensorflow import keras
from keras.applications.resnet import ResNet50
from keras.layers import GlobalAveragePooling2D, Dense, Flatten
import numpy as np

# Baseline Model

In [ ]:
(x_train , y_train) , (x_test , y_test) = keras.datasets.cifar10.load_data()
Y_train = tf.keras.utils.to_categorical(y_train, 10)
Y_test = tf.keras.utils.to_categorical(y_test, 10)
X_train = x_train / 255.0
X_test = x_test / 255.0

170508288/170498071 [==============================] - 13s 0us/step


In [ ]:
resnet = ResNet50(

    include_top=False, # classification : True, embedding : False
    weights= 'imagenet',
    input_shape=(32,32,3),
    pooling = 'avg', #global avarage pooling
    classes=10,
    classifier_activation ='softmax'
)
flattn_lyr = tf.keras.layers.Flatten()(resnet.output)
dense_1 = tf.keras.layers.Dense(1024, activation="relu")(flattn_lyr)
dense_2 = tf.keras.layers.Dense(512, activation="relu")(dense_1)
output = tf.keras.layers.Dense(10, activation="softmax", name="classification")(dense_2)
model = tf.keras.Model(inputs=resnet.input, outputs = output)


94781440/94765736 [==============================] - 0s 0us/step


In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])



In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 38, 38, 3)    0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 16, 16, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 16, 16, 64)   256         ['conv1_conv[0][0]']             
                                                                                              

In [ ]:
model.fit(
  x=X_train,
  y=Y_train,
  batch_size=500,
  epochs=5,
  validation_split=0.1
)

Epoch 1/5
90/90 [==============================] - 29s 191ms/step - loss: 1.1685 - accuracy: 0.6149 - val_loss: 3.6587 - val_accuracy: 0.1304
Epoch 2/5
90/90 [==============================] - 17s 184ms/step - loss: 0.6249 - accuracy: 0.7903 - val_loss: 3.3651 - val_accuracy: 0.0984
Epoch 3/5
90/90 [==============================] - 17s 186ms/step - loss: 0.4954 - accuracy: 0.8334 - val_loss: 2.9542 - val_accuracy: 0.0986
Epoch 4/5
90/90 [==============================] - 17s 186ms/step - loss: 0.3451 - accuracy: 0.8841 - val_loss: 4.0314 - val_accuracy: 0.1002
Epoch 5/5
90/90 [==============================] - 17s 185ms/step - loss: 0.2542 - accuracy: 0.9134 - val_loss: 3.2171 - val_accuracy: 0.1270


In [ ]:
_, baseline_model_accuracy = model.evaluate(
    X_test, Y_test, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)
model.save('model.h5')

Baseline test accuracy: 0.12359999865293503


In [ ]:
print(model.layers)

In [ ]:
for layer in model.layers: print(layer.get_config(), layer.get_weights())

Streaming output truncated to the last 5000 lines.
       1.0094845 , 0.9778967 , 1.2197227 , 0.8478163 , 0.82204324,
       1.2639296 , 0.96848184, 1.1698716 , 0.8272971 , 1.005974  ,
       0.86635834, 0.8941384 , 1.1798618 , 1.0616022 , 0.79700804,
       1.124055  , 0.94833463, 0.96091855, 1.0171818 , 1.2108041 ,
       1.1431711 , 0.7340896 , 1.0829633 , 1.0191426 , 1.065286  ,
       1.262183  , 1.3791144 , 0.8511555 , 0.86930764, 1.1351837 ,
       1.1387391 , 1.2145219 , 1.0032462 , 0.97440785, 1.104447  ,
       1.0027689 , 1.0132926 , 1.093061  , 0.9202979 , 1.2304075 ,
       1.0547091 , 0.8228202 , 0.80779284, 1.4465127 , 0.9884189 ,
       0.9660232 , 0.8864033 , 1.0414045 , 0.7861684 , 0.94325686,
       0.89190185, 0.9614594 , 0.9022455 , 0.9587305 , 0.72911847,
       1.0303382 , 1.5058894 , 0.93505925, 0.9279138 , 1.173263  ,
       0.92590934, 0.90476006, 0.8538672 , 1.274578  , 0.8149109 ,
       0.83509624, 0.86899483, 0.6958798 , 1.0169212 , 1.0944971 ,
       0.70

In [ ]:
first_layer_weights = model.layers[7].get_weights()[1]
# first_layer_biases  = model.layers[2].get_weights()[2]
# second_layer_weights = model.layers[1].get_weights()[0]
# second_layer_biases  = model.layers[1].get_weights()[1]

In [ ]:
print(first_layer_weights)

[ 1.4487217e-04  4.4271124e-05 -1.1105644e-04  2.1244630e-05
  8.1757418e-05 -1.2648255e-05  1.0398557e-04  5.9457609e-05
  2.7139758e-04 -1.8492610e-05 -9.2750131e-05  5.1934552e-05
 -1.1275095e-05  2.2779562e-04 -1.5008062e-04  1.4463514e-04
 -6.9832538e-05 -1.4287673e-07  1.3259962e-05  1.3183341e-05
 -8.5175612e-05 -1.1253953e-04  9.7099888e-05 -6.8033482e-06
 -4.4803553e-05 -5.9669901e-04  7.6169126e-05 -3.3636371e-04
 -1.0213380e-04  1.2302716e-04 -2.3305873e-04 -2.7568734e-05
  9.6512304e-05 -9.0041423e-05  2.4106375e-05 -3.6783014e-05
 -6.7544934e-05 -1.6730675e-05  8.8812694e-06  2.4925047e-04
  3.7481699e-05  1.3097505e-04  7.8952216e-06  1.2755161e-05
  1.5915101e-04 -6.2571351e-05  2.8399102e-04  2.3243756e-05
  2.6199874e-05 -2.2259188e-05 -2.4892068e-05  3.6987069e-04
  5.6657442e-05  1.6835985e-04  1.1705021e-04  7.0525079e-05
  5.0023853e-05 -2.9316088e-05  7.9227764e-05 -8.0831378e-05
  4.8161619e-06  3.0896082e-04 -2.4303912e-05  6.1301637e-04]


# Quantization with training

In [ ]:
(x_train , y_train) , (x_test , y_test) = keras.datasets.cifar10.load_data()
Y_train = tf.keras.utils.to_categorical(y_train, 10)
Y_test = tf.keras.utils.to_categorical(y_test, 10)
X_train = x_train / 255.0
X_test = x_test / 255.0


resnet = ResNet50(

    include_top=False, # classification : True, embedding : False
    weights= 'imagenet',
    input_shape=(32,32,3),
    pooling = 'avg', #global avarage pooling
    classes=10,
    classifier_activation ='softmax'
)
flattn_lyr = tf.keras.layers.Flatten()(resnet.output)
dense_1 = tf.keras.layers.Dense(1024, activation="relu")(flattn_lyr)
dense_2 = tf.keras.layers.Dense(512, activation="relu")(dense_1)
output = tf.keras.layers.Dense(10, activation="softmax", name="classification")(dense_2)
model = tf.keras.Model(inputs=resnet.input, outputs = output)


In [ ]:
import tensorflow_model_optimization as tfmot

quantize_model = tfmot.quantization.keras.quantize_model

# q_aware stands for for quantization aware.
q_aware_model = quantize_model(model)

# `quantize_model` requires a recompile.
q_aware_model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

q_aware_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 quantize_layer (QuantizeLayer)  (None, 32, 32, 3)   3           ['input_2[0][0]']                
                                                                                                  
 quant_conv1_pad (QuantizeWrapp  (None, 38, 38, 3)   1           ['quantize_layer[0][0]']         
 erV2)                                                                                            
                                                                                                  
 quant_conv1_conv (QuantizeWrap  (None, 16, 16, 64)  9601        ['quant_conv1_pad[0][0]']  

In [ ]:
q_aware_model.fit(
  x=X_train,
  y=Y_train,
  batch_size=500,
  epochs=20,
  validation_split=0.1
)

Epoch 1/20
90/90 [==============================] - 79s 755ms/step - loss: 1.2575 - accuracy: 0.5806 - val_loss: 8.0274 - val_accuracy: 0.1064
Epoch 2/20
90/90 [==============================] - 66s 733ms/step - loss: 0.6490 - accuracy: 0.7802 - val_loss: 3.1144 - val_accuracy: 0.1166
Epoch 3/20
90/90 [==============================] - 65s 727ms/step - loss: 0.4680 - accuracy: 0.8417 - val_loss: 2.8997 - val_accuracy: 0.1150
Epoch 4/20
90/90 [==============================] - 66s 732ms/step - loss: 0.3516 - accuracy: 0.8824 - val_loss: 2.7842 - val_accuracy: 0.0966
Epoch 5/20
90/90 [==============================] - 66s 728ms/step - loss: 0.2790 - accuracy: 0.9069 - val_loss: 3.4372 - val_accuracy: 0.1080
Epoch 6/20
90/90 [==============================] - 66s 734ms/step - loss: 0.2190 - accuracy: 0.9275 - val_loss: 3.4083 - val_accuracy: 0.1586
Epoch 7/20
90/90 [==============================] - 66s 728ms/step - loss: 0.1726 - accuracy: 0.9427 - val_loss: 3.1400 - val_accuracy: 0.2080

In [ ]:
baseline = tf.keras.models.load_model('model.h5')
print(baseline)
_, baseline_model_accuracy = baseline.evaluate(
    X_test, Y_test, verbose=1)

_, q_aware_model_accuracy = q_aware_model.evaluate(
   X_test, Y_test, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)
print('Quant test accuracy:', q_aware_model_accuracy)

313/313 [==============================] - 74s 225ms/step - loss: 5.5849 - accuracy: 0.1000
Baseline test accuracy: 0.12359999865293503
Quant test accuracy: 0.10000000149011612


In [ ]:
for layer in q_aware_model.layers: print(layer.get_config(), layer.get_weights())

Streaming output truncated to the last 5000 lines.
       -6., -6., -6., -6., -6., -6., -6., -6., -6., -6., -6., -6., -6.,
       -6., -6., -6., -6., -6., -6., -6., -6., -6., -6., -6., -6., -6.,
       -6., -6., -6., -6., -6., -6., -6., -6., -6., -6., -6., -6., -6.,
       -6., -6., -6., -6., -6., -6., -6., -6., -6., -6., -6., -6., -6.,
       -6., -6., -6., -6., -6., -6., -6., -6., -6., -6., -6., -6., -6.,
       -6., -6., -6., -6., -6., -6., -6., -6., -6., -6., -6., -6., -6.,
       -6., -6., -6., -6., -6., -6., -6., -6., -6., -6., -6., -6., -6.,
       -6., -6., -6., -6., -6., -6., -6., -6., -6., -6., -6., -6., -6.,
       -6., -6., -6., -6., -6., -6., -6., -6., -6., -6., -6., -6., -6.,
       -6., -6., -6., -6., -6., -6., -6., -6., -6., -6., -6., -6., -6.,
       -6., -6., -6., -6., -6., -6., -6., -6., -6., -6., -6., -6., -6.,
       -6., -6., -6., -6., -6., -6., -6., -6., -6., -6., -6., -6., -6.,
       -6., -6., -6., -6., -6., -6., -6., -6., -6., -6., -6., -6., -6.,
       -6., -

# Create quantized model for TFLite backend

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(q_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

quantized_tflite_model = converter.convert()

/usr/local/lib/python3.7/dist-packages/tensorflow/lite/python/convert.py:746: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


# See 4x smaller model from quantization

In [ ]:
# Create float TFLite model.
float_converter = tf.lite.TFLiteConverter.from_keras_model(model)
float_tflite_model = float_converter.convert()

# Measure sizes of models.
_, float_file = tempfile.mkstemp('.tflite')
_, quant_file = tempfile.mkstemp('.tflite')

with open(quant_file, 'wb') as f:
  f.write(quantized_tflite_model)

with open(float_file, 'wb') as f:
  f.write(float_tflite_model)

print("Float model in Mb:", os.path.getsize(float_file) / float(2**20))
print("Quantized model in Mb:", os.path.getsize(quant_file) / float(2**20))

Float model in Mb: 99.63154602050781
Quantized model in Mb: 25.638442993164062


# Clustering

In [ ]:
model= tf.keras.models.load_model('model.h5')


In [ ]:
cluster_weights = tfmot.clustering.keras.cluster_weights
CentroidInitialization = tfmot.clustering.keras.CentroidInitialization

clustering_params = {
  'number_of_clusters': 16,
  'cluster_centroids_init': CentroidInitialization.LINEAR
}

# Cluster a whole model
clustered_model = cluster_weights(model, **clustering_params)


clustered_model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy'])

clustered_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 cluster_conv1_pad (ClusterWeig  (None, 38, 38, 3)   0           ['input_1[0][0]']                
 hts)                                                                                             
                                                                                                  
 cluster_conv1_conv (ClusterWei  (None, 16, 16, 64)  18896       ['cluster_conv1_pad[0][0]']      
 ghts)                                                                                            
                                                                                              

In [ ]:
clustered_model.fit(
  X_train,
  Y_train,
  batch_size=500,
  epochs=20,
  validation_split=0.1)

Epoch 1/20
90/90 [==============================] - 44s 348ms/step - loss: 1.6632 - accuracy: 0.4685 - val_loss: 0.7861 - val_accuracy: 0.7386
Epoch 2/20
90/90 [==============================] - 29s 323ms/step - loss: 0.4663 - accuracy: 0.8424 - val_loss: 0.7220 - val_accuracy: 0.7742
Epoch 3/20
90/90 [==============================] - 29s 324ms/step - loss: 0.3509 - accuracy: 0.8803 - val_loss: 0.6823 - val_accuracy: 0.8046
Epoch 4/20
90/90 [==============================] - 29s 325ms/step - loss: 0.2174 - accuracy: 0.9254 - val_loss: 0.7728 - val_accuracy: 0.7922
Epoch 5/20
90/90 [==============================] - 29s 325ms/step - loss: 0.1958 - accuracy: 0.9327 - val_loss: 0.8592 - val_accuracy: 0.7650
Epoch 6/20
90/90 [==============================] - 29s 325ms/step - loss: 0.1750 - accuracy: 0.9406 - val_loss: 0.8037 - val_accuracy: 0.7950
Epoch 7/20
90/90 [==============================] - 29s 320ms/step - loss: 0.1428 - accuracy: 0.9512 - val_loss: 0.8984 - val_accuracy: 0.7912

In [ ]:
baseline = tf.keras.models.load_model('model.h5')
print(baseline)
_, clustered_model_accuracy = clustered_model.evaluate(
  X_test, Y_test, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)
print('Clustered test accuracy:', clustered_model_accuracy)


Baseline test accuracy: 0.7717999815940857
Clustered test accuracy: 0.7853000164031982


In [ ]:
for layer in clustered_model.layers: print(layer.get_config(), layer.get_weights())

Streaming output truncated to the last 5000 lines.
       1.3347728 , 1.4151517 , 1.2825642 , 0.81395984, 0.9875505 ,
       1.2061716 , 0.6631244 , 1.0881774 , 0.86748695, 1.066237  ,
       2.032378  , 0.7602033 , 1.1169794 , 0.9069455 , 0.885768  ,
       0.8640239 , 0.76216775, 1.1768005 , 1.3805972 , 1.2042733 ,
       1.0311636 , 1.0570794 , 0.99590456, 1.6378261 , 0.81064254,
       1.2365978 , 1.7745969 , 1.0997894 , 0.96963364, 1.2099121 ,
       1.1990874 , 1.171676  , 1.3388493 , 1.0948777 , 1.2790701 ,
       0.72490567, 1.7515541 , 0.7778958 , 1.2173641 , 0.97148204,
       0.9106862 , 0.91701317, 1.5788083 , 1.3427907 , 1.014453  ,
       1.2682548 , 1.3159484 , 1.1279017 , 1.4115916 , 0.6987862 ,
       1.3905823 , 0.8218755 , 0.99731773, 1.1462774 , 1.1549925 ,
       1.4094762 , 0.60771626, 1.6867813 , 1.5100899 , 1.1346298 ,
       1.1704359 , 1.9171647 , 1.4125968 , 0.7758243 , 1.4245447 ,
       1.1565195 , 1.0046425 , 0.89405805, 2.4557214 , 0.6969785 ,
       0.78

# Create 6x smaller models from clustering

In [ ]:
final_model = tfmot.clustering.keras.strip_clustering(clustered_model)

_, clustered_keras_file = tempfile.mkstemp('model.h5')
print('Saving clustered model to: ', clustered_keras_file)
tf.keras.models.save_model(final_model, clustered_keras_file,
                           include_optimizer=False)

Saving clustered model to:  /tmp/tmpwdzcdeq4model.h5


In [ ]:
clustered_tflite_file = '/tmp/clustered_cifar10.tflite'
converter = tf.lite.TFLiteConverter.from_keras_model(final_model)
tflite_clustered_model = converter.convert()
with open(clustered_tflite_file, 'wb') as f:
  f.write(tflite_clustered_model)
print('Saved clustered TFLite model to:', clustered_tflite_file)

Saved clustered TFLite model to: /tmp/clustered_cifar10.tflite


# Define a helper function to actually compress the models via gzip and measure the zipped size.

In [ ]:
def get_gzipped_model_size(file):
  # It returns the size of the gzipped model in bytes.
  import os
  import zipfile

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)

# Compare and see that the models are 6x smaller from clustering

In [ ]:
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size('model.h5')))
print("Size of gzipped clustered Keras model: %.2f bytes" % (get_gzipped_model_size(clustered_keras_file)))
print("Size of gzipped clustered TFlite model: %.2f bytes" % (get_gzipped_model_size(clustered_tflite_file)))

Size of gzipped baseline Keras model: 240127854.00 bytes
Size of gzipped clustered Keras model: 11359620.00 bytes
Size of gzipped clustered TFlite model: 12203465.00 bytes


# Create an 8x smaller TFLite model from combining weight clustering and post-training quantization
# You can apply post-training quantization to the clustered model for additional benefits.

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(final_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()



with open('tflite_quant_model.tflite', 'wb') as f:
  f.write(tflite_quant_model)

print('Saved quantized and clustered TFLite model to:', 'tflite_quant_model.tflite')
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size('model.h5')))
print("Size of gzipped clustered and quantized TFlite model: %.2f bytes" % (get_gzipped_model_size('tflite_quant_model.tflite')))

Saved quantized and clustered TFLite model to: tflite_quant_model.tflite
Size of gzipped baseline Keras model: 240127854.00 bytes
Size of gzipped clustered and quantized TFlite model: 8927610.00 bytes


In [ ]:
interpreter = tf.lite.Interpreter(model_path = 'tflite_quant_model.tflite')
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("Input Shape:", input_details[0]['shape'])
print("Input Type:", input_details[0]['dtype'])
print("Output Shape:", output_details[0]['shape'])
print("Output Type:", output_details[0]['dtype'])

Input Shape: [ 1 32 32  3]
Input Type: <class 'numpy.float32'>
Output Shape: [ 1 10]
Output Type: <class 'numpy.float32'>


In [ ]:
interpreter.resize_tensor_input(input_details[0]['index'],(10000, 32, 32, 3))
interpreter.resize_tensor_input(output_details[0]['index'],(10000, 10))
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("Input Shape:", input_details[0]['shape'])
print("Input Type:", input_details[0]['dtype'])
print("Output Shape:", output_details[0]['shape'])
print("Output Type:", output_details[0]['dtype'])

Input Shape: [10000    32    32     3]
Input Type: <class 'numpy.float32'>
Output Shape: [10000    10]
Output Type: <class 'numpy.float32'>


In [ ]:
X_test.dtype

dtype('float64')

In [ ]:
X_test_numpy = np.array(X_test, dtype=np.float32)

In [ ]:
interpreter.set_tensor(input_details[0]['index'],X_test_numpy)
interpreter.invoke()
tflite_model_predictions = interpreter.get_tensor(output_details[0]['index'])
prediction_classes = np.argmax(tflite_model_predictions, axis=1)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
print(prediction_classes)

[3 8 8 ... 5 5 7]


In [ ]:
print(Y_test)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]]


In [ ]:
print(y_test)

[[3]
 [8]
 [8]
 ...
 [5]
 [1]
 [7]]


In [ ]:
acc = accuracy_score(prediction_classes, y_test)

In [ ]:
print(acc)

0.7723


Pruning

In [ ]:
model= tf.keras.models.load_model('model.h5')

In [ ]:
import tensorflow_model_optimization as tfmot
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Compute end step to finish pruning after 12 epochs.
batch_size = 500
epochs = 20
validation_split = 0.1 # 10% of training set will be used for validation set.

num_images = X_train.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

# Define model for pruning.
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
}

model_for_pruning = prune_low_magnitude(model, **pruning_params)

# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model_for_pruning.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 prune_low_magnitude_conv1_pad   (None, 38, 38, 3)   1           ['input_1[0][0]']                
 (PruneLowMagnitude)                                                                              
                                                                                                  
 prune_low_magnitude_conv1_conv  (None, 16, 16, 64)  18882       ['prune_low_magnitude_conv1_pad[0
  (PruneLowMagnitude)                                            ][0]']                           
                                                                                              

In [ ]:
model_for_pruning.fit(
  X_train,
  Y_train,
  batch_size=500,
  epochs=20,
  validation_split=0.1)

Epoch 1/20
90/90 [==============================] - 28s 238ms/step - loss: 0.1035 - accuracy: 0.9686 - val_loss: 1.4117 - val_accuracy: 0.7644
Epoch 2/20
90/90 [==============================] - 20s 217ms/step - loss: 0.0550 - accuracy: 0.9829 - val_loss: 0.9400 - val_accuracy: 0.7986
Epoch 3/20
90/90 [==============================] - 20s 217ms/step - loss: 0.0384 - accuracy: 0.9882 - val_loss: 0.9710 - val_accuracy: 0.8032
Epoch 4/20
90/90 [==============================] - 20s 222ms/step - loss: 0.0404 - accuracy: 0.9870 - val_loss: 0.9972 - val_accuracy: 0.8014
Epoch 5/20
90/90 [==============================] - 19s 216ms/step - loss: 0.0362 - accuracy: 0.9890 - val_loss: 0.9687 - val_accuracy: 0.7952
Epoch 6/20
90/90 [==============================] - 19s 215ms/step - loss: 0.0350 - accuracy: 0.9889 - val_loss: 1.0132 - val_accuracy: 0.7968
Epoch 7/20
90/90 [==============================] - 19s 216ms/step - loss: 0.0353 - accuracy: 0.9888 - val_loss: 1.0467 - val_accuracy: 0.7872

In [ ]:
baseline = tf.keras.models.load_model('model.h5')
print(baseline)
_, model_for_pruning_accuracy = model_for_pruning.evaluate(
   X_test, Y_test, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)
print('Pruned test accuracy:', model_for_pruning_accuracy)

Baseline test accuracy: 0.7717999815940857
Pruned test accuracy: 0.792900025844574


# Create 3x smaller models from pruning

In [ ]:
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)

_, pruned_keras_file = tempfile.mkstemp('model.h5')
tf.keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)
print('Saved pruned Keras model to:', pruned_keras_file)

Saved pruned Keras model to: /tmp/tmpnvcyvitbmodel.h5


# Then, create a compressible model for TFLite.

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
pruned_tflite_model = converter.convert()

_, pruned_tflite_file = tempfile.mkstemp('.tflite')

with open(pruned_tflite_file, 'wb') as f:
  f.write(pruned_tflite_model)

print('Saved pruned TFLite model to:', pruned_tflite_file)

Saved pruned TFLite model to: /tmp/tmpi5wm1zlg.tflite


# Define a helper function to actually compress the models via gzip and measure the zipped size.

In [ ]:
def get_gzipped_model_size(file):
  # Returns size of gzipped model, in bytes.
  import os
  import zipfile

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)

# Compare and see that the models are 3x smaller from pruning.

In [ ]:
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size('model.h5')))
print("Size of gzipped pruned Keras model: %.2f bytes" % (get_gzipped_model_size(pruned_keras_file)))
print("Size of gzipped pruned TFlite model: %.2f bytes" % (get_gzipped_model_size(pruned_tflite_file)))

Size of gzipped baseline Keras model: 240127811.00 bytes
Size of gzipped pruned Keras model: 31410725.00 bytes
Size of gzipped pruned TFlite model: 30892150.00 bytes


# Create a 10x smaller model from combining pruning and quantization

You can apply post-training quantization to the pruned model for additional benefits

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_and_pruned_tflite_model = converter.convert()


with open('quantized_and_pruned_tflite_file.tflite', 'wb') as f:
  f.write(quantized_and_pruned_tflite_model)

print('Saved quantized and pruned TFLite model to:', 'quantized_and_pruned_tflite_file.tflite')

print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size('model.h5')))
print("Size of gzipped pruned and quantized TFlite model: %.2f bytes" % (get_gzipped_model_size('quantized_and_pruned_tflite_file.tflite')))

Saved quantized and pruned TFLite model to: quantized_and_pruned_tflite_file.tflite
Size of gzipped baseline Keras model: 240127811.00 bytes
Size of gzipped pruned and quantized TFlite model: 9450299.00 bytes


In [ ]:
interpreter = tf.lite.Interpreter(model_path = 'quantized_and_pruned_tflite_file.tflite')
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("Input Shape:", input_details[0]['shape'])
print("Input Type:", input_details[0]['dtype'])
print("Output Shape:", output_details[0]['shape'])
print("Output Type:", output_details[0]['dtype'])

Input Shape: [ 1 32 32  3]
Input Type: <class 'numpy.float32'>
Output Shape: [ 1 10]
Output Type: <class 'numpy.float32'>


In [ ]:
interpreter.resize_tensor_input(input_details[0]['index'],(10000, 32, 32, 3))
interpreter.resize_tensor_input(output_details[0]['index'],(10000, 10))
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("Input Shape:", input_details[0]['shape'])
print("Input Type:", input_details[0]['dtype'])
print("Output Shape:", output_details[0]['shape'])
print("Output Type:", output_details[0]['dtype'])

Input Shape: [10000    32    32     3]
Input Type: <class 'numpy.float32'>
Output Shape: [10000    10]
Output Type: <class 'numpy.float32'>


In [ ]:
X_test.dtype

dtype('float64')

In [ ]:
X_test_numpy = np.array(X_test, dtype=np.float32)

In [ ]:
interpreter.set_tensor(input_details[0]['index'],X_test_numpy)
interpreter.invoke()
tflite_model_predictions = interpreter.get_tensor(output_details[0]['index'])
prediction_classes = np.argmax(tflite_model_predictions, axis=1)

In [ ]:
acc = accuracy_score(prediction_classes, y_test)

In [ ]:
print(acc)

0.7945
